# Feature extraction

In [5]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [6]:
import pandas as pd
import feature_extraction

## Load dataset

In [7]:
# Load CB16 dataset
df = pd.read_json("../data/clickbait17-train-170331/instances.jsonl", lines=True, encoding='utf8')
df.set_index("id", inplace=True)
df_raw = df.copy()

df_truth = pd.read_json("../data/clickbait17-train-170331/truth.jsonl", lines=True, encoding='utf8')
df_truth.set_index("id", inplace=True)
df_truth_raw = df_truth.copy()

In [8]:
# Add truth judgments to df
df = df.join(df_truth[["truthClass"]]) # also available: truthClass, truthJudgments, truthMean, truthMedian, truthMode

In [10]:
df.head(2)
len(df)

2459

## Extract features

In [13]:
fe = feature_extraction.FeatureExtractor(r"..\data\clickbait17-train-170331", None) 
fe.set_df(df)

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\kaspe\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\wordnet.zip.


In [14]:
labels, features = fe.extract_features()

In [21]:
features.join(labels)
features[:2]

AttributeError: 'numpy.int64' object has no attribute 'index'

## Classification stuff

In [77]:
import classification
from sklearn.ensemble import RandomForestClassifier

# Define the classifiers and the grid that we want to optimize on
# Note: either define a grid and call optimize, or supply optimized_param
classifiers = [
    {
        'name': 'RandomForest',
        'clf': RandomForestClassifier(),
#         'grid': {
#             'n_estimators': [100, 1000],
#             'max_depth': [2, 3]
#         },
        'optimized_param':{
            'max_depth': 3,
            'n_estimators': 1000
        }
    },
]

classi = classification.Classifiers(features,labels, classifiers)

In [79]:
classi.information_gain()
# TODO: chi2 requires non-negative feature values -> we have -1 in the set
# classi.chi2_stats()

Information gain of whole dataset
[0.02264157 0.01396756 0.         0.01204188 0.00319258 0.01098833
 0.01329451 0.0056955  0.02341842 0.01216696 0.00331194 0.
 0.02253567 0.03808076 0.00883827]


array([0.02264157, 0.01396756, 0.        , 0.01204188, 0.00319258,
       0.01098833, 0.01329451, 0.0056955 , 0.02341842, 0.01216696,
       0.00331194, 0.        , 0.02253567, 0.03808076, 0.00883827])

In [81]:
# classi.optimize()
classi.cross_val()
classi.test()

-- Cross validation with 10-folds --
Cross validation performance:
{'fit_time': 1.071239161491394, 'score_time': 0.25610487461090087, 'test_accuracy': 0.7084071677451469, 'train_accuracy': 0.7170032463993867, 'test_precision': 0.769065934065934, 'train_precision': 0.8475789617903111, 'test_f1': 0.15567431599712628, 'train_f1': 0.18805817147187523, 'test_roc_auc': 0.6791747103208972, 'train_roc_auc': 0.7233079954886652, 'test_recall': 0.08779999163960452, 'train_recall': 0.10583236176849761}
-- Finished cross validation --
-- Performance on split: 80% train - 20% split --
              precision    recall  f1-score   support

     class 0       0.70      0.97      0.82       339
     class 1       0.61      0.09      0.16       153

   micro avg       0.70      0.70      0.70       492
   macro avg       0.66      0.53      0.49       492
weighted avg       0.67      0.70      0.61       492

-- Finished test reports --
